In [1]:
# This notebook checks MNE functions for ECG and EOG detection
# Cropped data is used here (5 minutes only), tried on whole data - takes forever.


#Load data, make folders
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import mne
from mne.time_frequency import tfr_morlet, psd_multitaper, psd_welch
import configparser
import ancpbids

from main_meg_qc import make_derivative_html, sanity_check, initial_stuff
from RMSE_meq_qc import MEG_QC_rmse
from PSD_meg_qc import PSD_QC 


In [2]:
#%% TRY TO SAVE DERIVATIVE WITH BIDS DATASET HERE:
make_derivative_html()

Opening raw data file /Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/data/ds004276/sub-001/meg/sub-001_task-words_meg.fif...
    Range : 0 ... 467999 =      0.000 ...   467.999 secs
Ready.
No events with set minimum duration were found using all stimulus channels. No epoching can be done. Try different event duration in config file.
There are no gradiometers in this data set: check parameter do_for in config file. Analysis will be done only for magnetometers.


Magnetometers
  
Average absolute power per band:
delta 1.1633824715959679e-26
theta 2.53612252689823e-27
alpha 1.1898899884077815e-27
beta 2.338060073810511e-27
gamma 1.1036129486365862e-27
  
Average relative power per band:
delta 0.4164420618767332
theta 0.1530678570600671
alpha 0.0986540307800646
beta 0.2195700962715598
gamma 0.1133036422548931
  
Average power/freq per band:
delta 2.9084561789899198e-27
theta 5.072245053796461e-28
alpha 2.379779976815563e-28
beta 1.2305579335844794e-28
gamma 1.554384434699417e-29


FIGURE! Figure({
    'data': [{'hovertemplate': 'Frequency: %{x} Hz<br>Amplitude: %{y: .2e} T/Hz',
              'name': 'MEG 001',
              'type': 'scatter',
              'x': array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,  12.,
                           13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,  23.,  24.,
                           25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,  34.,  35.,  36.,
                           37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,  45.,  46.,  47.,  48.,
                           49.,  50.,  51.,  52.,  53.,  54.,  55.,  56.,  57.,  58.,  59.,  60.,
                           61.,  62.,  63.,  64.,  65.,  66.,  67.,  68.,  69.,  70.,  71.,  72.,
                           73.,  74.,  75.,  76.,  77.,  78.,  79.,  80.,  81.,  82.,  83.,  84.,
                           85.,  86.,  87.,  88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,
                           97.,  98.,  99., 100.]),

In [ ]:
# Other useful ancp stuff:

config = configparser.ConfigParser()
config.read('settings.ini')

default_direct = config['DEFAULT']['data_directory']
dataset_path = ancpbids.utils.fetch_dataset(default_direct)

from ancpbids import BIDSLayout
layout = BIDSLayout(dataset_path)

list_of_fifs = layout.get(suffix='meg', extension='.fif', return_type='filename')

list_of_subs = layout.get_subjects()

list_of_entities = layout.get_entities()
print(list_of_entities)

In [2]:
#%% TRY SEPARATE FUNCTIONS HERE

config = configparser.ConfigParser()
config.read('settings.ini')
data_file='/Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/data/sub_HT05ND16/210811/mikado-1.fif'
#data_file='/Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/data/ds004276/sub-001/meg/sub-001_task-words_meg.fif'
sid='001'

df_epochs, epochs_mg, channels, raw_bandpass, raw_bandpass_resamp, raw_cropped, raw = initial_stuff(config, data_file)
m_or_g_chosen = ['mags', 'grads']

m_or_g_chosen = sanity_check(m_or_g_chosen, channels)

if len(m_or_g_chosen) == 0: 
    raise ValueError('No channels to analyze. Check presence of mags and grads in your data set and parameter do_for in settings.')




Opening raw data file /Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/data/sub_HT05ND16/210811/mikado-1.fif...
    Read a total of 8 projection items:
        magn8_iasoff_68deg.fif : PCA-v1 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v2 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v3 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v4 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v5 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v6 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v7 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v8 (1 x 306)  idle
    Range : 1809000 ... 3375999 =   1809.000 ...  3375.999 secs
Ready.
Opening raw data file /Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/data/sub_HT05ND16/210811/mikado-2.fif...
    Read a total of 8 projection items:
        magn8_iasoff_68deg.fif : PCA-v1 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v2 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v3 (1 x 30

/Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/Functions/main_meg_qc.py:45: RuntimeWarning: This filename (/Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/data/sub_HT05ND16/210811/mikado-1.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(data_file)


        magn8_iasoff_68deg.fif : PCA-v7 (1 x 306)  idle
        magn8_iasoff_68deg.fif : PCA-v8 (1 x 306)  idle
    Range : 4943000 ... 5198999 =   4943.000 ...  5198.999 secs
Ready.
Trigger channel has a non-zero initial value of 18 (consider using initial_event=True to detect this event)
9 events found
Event IDs: [ 9 19 20 21 22 23]
Not setting metadata
9 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 8)
8 projection items activated
Loading data for 9 events and 1201 original time points ...
0 bad epochs dropped
Not setting metadata
9 matching events found
No baseline correction applied
8 projection items activated
Loading data for 9 events and 1201 original time points ...
0 bad epochs dropped


/Users/jenya/Local Storage/Job Uni Rieger lab/MEG QC code/Functions/data_load_and_folders.py:118: RuntimeWarning: Trigger channel contains negative values, using absolute value. If data were acquired on a Neuromag system with STI016 active, consider using uint_cast=True to work around an acquisition bug
  events = mne.find_events(data, stim_channel=stim_channel, min_duration=event_dur)


In [3]:
list_of_figures, _, list_of_fig_desc = MEG_QC_rmse(sid, config, channels, df_epochs, raw_bandpass_resamp, m_or_g_chosen)

#from PSD_meg_qc import Freq_Spectrum_meg

#freqs, psds, fig_path_psd, fig_psd, fig_desc = Freq_Spectrum_meg(data=raw_bandpass_resamp, m_or_g = 'mags', sid=sid, freq_min=0.5, freq_max=100, n_fft=1000, n_per_seg=1000, freq_tmin=None, freq_tmax=None, ch_names=channels['mags'])

[1.60613831e-12 1.53055999e-12 1.52248010e-12 1.58577513e-12
 1.46839288e-12 1.46325968e-12 1.43100048e-12 1.45712233e-12
 1.48478907e-12 1.48856224e-12 1.41007143e-12 1.49704637e-12
 1.36429439e-12 1.20774766e-12 1.14851100e-12 1.32185325e-12
 1.48630962e-12 1.45508255e-12 1.43060900e-12 1.49774995e-12
 1.33807100e-12 1.08753389e-12 1.02559581e-12 1.23411637e-12
 9.53037852e-13 8.53385460e-13 8.39331965e-13 9.14756489e-13
 1.40965243e-12 1.36795511e-12 1.36941783e-12 1.31997288e-12
 1.19654039e-12 1.30116502e-12 1.25332064e-12 1.15605990e-12
 1.01412081e-12 9.29968000e-13 8.98647278e-13 1.03969079e-12
 1.04133245e-12 8.74851100e-13 1.30605609e-12 1.32279518e-12
 1.17417140e-12 1.04934563e-12 1.19539659e-12 1.32971436e-12
 1.36470778e-12 1.22580079e-12 1.42423358e-12 1.45344150e-12
 1.44434151e-12 1.40530090e-12 1.47936317e-12 1.46207068e-12
 1.55287371e-12 1.55526895e-12 1.43862154e-12 1.39962036e-12
 1.35359278e-12 1.41900771e-12 1.57289690e-12 1.45007551e-12
 1.48208647e-12 1.624378

[9.11584084e-12 1.01650186e-11 8.88658557e-12 8.10188443e-12
 9.20099448e-12 7.80211464e-12 8.00356145e-12 9.27597281e-12
 6.86609588e-12 7.96353998e-12 7.95219539e-12 9.59680936e-12
 7.77142380e-12 6.50740602e-12 7.84064958e-12 7.31727605e-12
 7.48674028e-12 8.01072516e-12 7.27956964e-12 6.25970798e-12
 5.98609913e-12 6.32811826e-12 6.61722903e-12 8.35275238e-12
 6.19903245e-12 6.40824584e-12 7.50812824e-12 6.55223550e-12
 7.04324467e-12 6.15206631e-12 6.88029850e-12 6.73879535e-12
 6.04934897e-12 6.43325863e-12 6.51715721e-12 7.16079475e-12
 7.05898592e-12 6.01211201e-12 7.24385106e-12 6.00217039e-12
 5.79442409e-12 6.13133351e-12 6.76192971e-12 6.20327316e-12
 5.66302973e-12 6.10379688e-12 6.98783669e-12 6.09992188e-12
 6.08181666e-12 6.17447700e-12 7.00430873e-12 6.77179074e-12
 5.79362320e-12 5.65671702e-12 5.65819998e-12 5.73629435e-12
 6.29991998e-12 9.13254873e-12 8.98882764e-12 5.78993804e-12
 5.97771257e-12 7.44123137e-12 6.51151441e-12 7.08724744e-12
 6.45843810e-12 6.017542